# Lab 5: Randomized Response

***
- **FIRST name**: Abimbola
- **LAST name**: Olarinde
- **Student ID**: 1880229

Leave blank if individual:
- **Collaborator names**:
- **Collaborator student IDs**:
***

In today's lab, you will learn:

1. apply randomized response mechanisms;
2. obtain the estimated count of true responses;
3. apply biased randomized response mechanisms;
4. calculate the relative error.

For this lab, you'll need the dataset `adult.csv`.

### Instructions

- **Collaboration**: You must submit your own work. The collaboration policy for the labs is Consultation Collaboration. You may verbally discuss concepts with your classmates, without exchanging written text, code, or detailed advice. You must develop your own solution and submit your own work. All sources of information used including books, websites, students you talked to, must be cited in the submission. Please see the course FAQ document for details on this collaboration policy. We will adhere to current Faculty of Science guidelines on dealing with suspected cases of plagiarism.
- **Software**: We highly recommend that students use Google Colab for completing labs and assignments. This is the software used by the TAs in the course, and we can guarantee that there will be no issues with incompatible environments or imports.
- **Filling out the Notebook**: You must use this notebook to complete your lab. You will execute the questions in the notebook. The questions might ask for a short answer in text form or for you to write and execute a piece of code. Make sure you enter your answer in either case only in the cell provided.
- **Important**:  Do not use a different cell, do not delete cells, and do not create a new cell. Creating new cells for your code is not compatible with the auto-grading system we are using and thus your assignment will not get grading properly and you will lose marks for that question. As a reminder you must remove the raise NotImplementedError() statements from each question when answering.
- **Rules for Datasets**: Any datasets used in the lab cannot be imported from cloud storage, e.g google drive, and must be read from a file either on your local computer or uploaded to the google colab notebook. Importing from cloud storage will result in a zero.
- **Submission Formatting**: When you are done, you will submit your work from the notebook. Make sure to save your notebook before running it, and then submit on Canvas the notebook file with your work completed. Name your file with your Student ID number, followed by an underscore and L plus the lab number (ex: 1234567_L5.ipynb). Failure to do so will result in your final score being reduced by 50%! Finally your name must be written at the top of the lab or assignment document.

In [79]:
# Don't change this cell; just run it.
import numpy as np
from numpy.random import default_rng
rng = default_rng()
import pandas as pd
from scipy.optimize import minimize

# These lines do some fancy plotting magic.
import matplotlib
# This is a magic function that renders the figure in the notebook, instead of displaying a dump of the figure object.
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

## Importing and Cleaning Data


### Question 1
Question 1.1. Load the dataset `adult.csv` into a pandas dataframe and drop the missing values.

In [80]:
# YOUR CODE HERE
adult = pd.read_csv('adult.csv')
adult.dropna(inplace=True)
adult.head()


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K


In [81]:
#TEST CELL: do not delete!

Question 1.2. Now convert the values in the income column to 1 (>50K) and 0 (<=50K)

In [82]:
# YOUR CODE HERE

adult["income"] = adult["income"].apply(lambda x: 1 if x == ">50K" else 0)
adult


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,0
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,0
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,0
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,0
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,0
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0


In [83]:
#TEST CELL: do not delete!

## Randomized Response

Now let's implement a Randomized Response mechanism.  Let's assume the query is about the income being <=50K or >50K.  The value in the `income` column is the true answer.

### Question 2
Question 2.1. Consider the following randomized response to the query on each row of whether the income is <=50K or >50K.  The reported answer is as follows:

Flip a fair coin.
- If it's heads, respond truthfully:  the reported answer is the same as the value in `income`.
- If it's tails, flip a fair coin again:
   - If it's heads, the reported answer has a value of 1.
   - If it's tails, the reported answer has a value of 0.
  
Implement a function for this mechanism in the code cell below.
The function should accept a value `0` or `1` and return as output the reported answer according to the randomized response above.

In [84]:
import random
def rand_resp_c1(x):
    """
    Implements the randomized response mechanism for income data.
    
    Parameters:
    income (int): The true income category (0 for <=50K, 1 for >50K).
    
    Returns:
    int: The reported income after applying the randomized response mechanism

    """
    if random.randint(0, 1) == 1:
        return x  # Report the true value
    
    # Second coin flip  if first was tails
    return random.randint(0, 1)  # Randomly return 0 or 1
    

adult['income_rrc1'] = [rand_resp_c1(x) for x in adult['income']]
adult['income_rrc1']

0        1
1        0
2        1
3        0
4        0
        ..
32556    0
32557    0
32558    1
32559    1
32560    1
Name: income_rrc1, Length: 32561, dtype: int64

In [85]:
#TEST CELL: do not delete!

### Question 3
Question 3.1. Now get the **estimate** for the true number of people with income >50K.  Write this result into the variable `est_true_yes`.  Given the number of reported `1`'s (income >50K), we know how to estimate the proportion or number of true `1`'s.

Calculate the true number of people with income >50K (the true answer in the data we imported) and write it into the variable `true_yes`.

**Hint**: recall the derivations we did in class to analyse our randomized response:
- $n$ is the total number of participants
- $z$ is the sum of obfuscated answers
- $m$ is the sum of true answers

<p> Since we usually don't have access to $m$, how could we estimate it using $n$ and $z$? </p>

In [86]:
# YOUR CODE HERE
# Compute the estimates
n = len(adult)  # Total number of participants
z = adult['income_rrc1'].sum()  # Sum of reported '1's
m = adult['income'].sum()  # True count of '1's

# Estimate of true '1's (income >50K)
est_true_yes = (2 * z - 0.5 * n)
true_yes = m

print(f"Estimated true count of income >50K: {est_true_yes}")
print(f"Actual true count of income >50K: {true_yes}")



Estimated true count of income >50K: 7717.5
Actual true count of income >50K: 7841


In [87]:
#TEST CELL: do not delete!

Question 3.2. Now consider a more general randomized response mechanism to the query on each row of whether the income is <=50K or >50K. This more general mechanism, that takes a parameter `bias` as an argument, is as follows:


With probability `1/2 + bias`, respond truthfully.
With probability `1/2 - bias`, respond with the opposite of the true answer.

Implement a function for this mechanism.


In [88]:
def rand_resp_cg(x, b):
    # YOUR CODE HERE
    """
    Randomized response mechanism.
    
    Parameters:
    x (int): The true value (0 or 1).
    b (float): The bias parameter (0 <= b <= 0.5).
    
    Returns:
    int: The randomized response (0 or 1).
    """
    if not (0 <= b <= 0.5):
        print("Bias must be between 0 and 0.5")
    
    prob = 0.5 + b
    if random.random() < prob:
        return x  # Respond truthfully
    else:
        return 1 - x  # Respond with the opposite of the true answer


In [89]:
#TEST CELL: do not delete!

Now as we did for the coin flip case we want to compare the estimated number of people with income >50K (the responses from the randomized response) with the true number (from the dataset - the `income` column).  The difference between the two values is the error in estimation.

For this generalized randomized response function, we have a parameter `b`, the bias, that is a part of the probability of responding truthfully or not - so it is a privacy parameter.  We want to analyze the error in the estimate of the number of people with income >50K, with respect to the privacy parameter, `b`.  

Question 3.3. So let's first set up an array, that we will call `bias_values`, to hold various values of `bias` that we will test, `bias` from 0.05 to 0.45.  Let's say we want to test for bias from 0.05 to 0.45 in increments of 0.05.

In [90]:
# YOUR CODE HERE
bias_values = np.arange(0.05, 0.50, 0.05)
bias_values

array([0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45])

In [91]:
#TEST CELL: do not delete!

### Question 4
Question 4.1. Now write code to calculate the estimated number of true `1` answers for each value of `bias`.  That is, you want to run your function above, `rand_resp_cg(x,b)` on the `income` column (`x`) and for the above values of bias (`b`) in the `bias_values` array.  The 2d array `rrg` will hold the randomized response values for the `income` column for each bias value in the `bias_values` array.  So, the value of `rrg[i]` will be an array of randomized responses from the `income` column that is output from `rand_resp_cg` for bias value `bias_values[i]`.  So, the 2d array `rrg` should have dimensions "number of bias values you are testing" and "number of rows in the cleaned up dataset".

After the first two lines of code in the cell below, write your code to fill `rrg` with the randomized responses and `est_yes` will contain the *estimated* number of true `1` responses for each value of bias.  So `est_yes[0]` will contain the estimated number of true yes responses when the bias is 0.05.

**Hint** : this is similar to calculating `est_true_yes` in Question 2, however now you have to consider how to include the bias in your derivation.

In [92]:

# YOUR CODE HERE
# Initializing  the arrays
rrg = np.zeros((len(bias_values), len(adult['income'])))
est_yes = np.zeros(len(bias_values))

# Fill rrg with randomized responses and calculate est_yes
for i, b in enumerate(bias_values):
    rrg[i] = adult['income'].apply(lambda x: rand_resp_cg(x, b))
    est_yes[i] = (np.sum(rrg[i]) - (0.5 - b) * len(adult['income'])) / (2 * b)

# Display the results
rrg, est_yes

(array([[1., 0., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 0., ..., 1., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 1.],
        [0., 0., 0., ..., 1., 0., 1.],
        [0., 0., 0., ..., 1., 0., 0.]]),
 array([8625.5       , 7593.        , 7598.83333333, 8069.25      ,
        7955.5       , 7656.33333333, 7954.07142857, 7838.625     ,
        7827.72222222]))

In [93]:
#TEST CELL: do not delete!

Question 4.2. Implement a function called `rel_error` to calculate the relative error between the true number of `1`s (in the `income` column) and the estimated number (relative to the true number).  The function should accept arrays as input and output an array of relative errors.  (So it should be able to accept arrays of numbers and output an array of the relative errors.)

In [94]:
def rel_error(true,estim):
    # YOUR CODE HERE
    return np.abs(true - estim) / true

print(rel_error(np.sum(rrg,1),est_yes))

[0.44405414 0.47789314 0.44436726 0.37909741 0.34349728 0.31061288
 0.23899049 0.17722001 0.09746083]


In [95]:
#TEST CELL: do not delete!

# Rubric

| Question | Points|
|----------|----------|
| 1.1   |  5  |
| 1.2    | 5   |
| 2.1    | 10   |
| 3.1    | 10   |
| 3.2  | 10   |
| 3.3    | 5   |
| 4.1   | 20   |
| 4.2   | 5   |
| Total:    | 70   |
